# Download dependices

In [1]:
!pip install hy3dgen
!pip install --upgrade huggingface_hub
!git clone https://github.com/Tencent/Hunyuan3D-2.git
!git clone https://github.com/kijai/ComfyUI-Hunyuan3DWrapper.git
!pip install -r  /content/ComfyUI-Hunyuan3DWrapper/requirements.txt
%cd /content/ComfyUI-Hunyuan3DWrapper/hy3dgen/texgen/custom_rasterizer/
!python setup.py bdist_wheel
!pip install dist/custom_rasterizer*.whl # Replace with actual filename
! python /content/Hunyuan3D-2/setup.py install
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 712.4/712.4 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.2/262.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Server

In [2]:
!pip install fastapi uvicorn aiofiles pyngrok python-multipart
!pip install trimesh torch torchvision
!pip install hy3dgen  # Make sure this is the correct package name

# Install ngrok
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
  && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list \
  && sudo apt update \
  && sudo apt install ngrok
!pip install fastapi uvicorn aiofiles pyngrok python-multipart trimesh torch


DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/hy3dgen-2.0.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/hy3dgen-2.0.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/hy3dgen-2.0.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get

In [3]:
!ngrok config add-authtoken "your_ngrok_authtoken"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [1]:

# === CELL 2: Setup and Imports ===
import os
import tempfile
import uuid
import asyncio
import logging
import threading
from pathlib import Path
from typing import Optional

from fastapi import FastAPI, File, UploadFile, HTTPException, BackgroundTasks
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import torch
import trimesh
from pyngrok import ngrok
import aiofiles
import nest_asyncio

# Enable nested asyncio (required for Colab)
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# === CELL 3: FastAPI App Setup ===
app = FastAPI(
    title="3D Model Generator API",
    description="Generate 3D models from images using Hunyuan3D (Colab Version)",
    version="1.0.0"
)

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Colab-specific paths
TEMP_DIR = Path("/content/temp_3d")
TEMP_DIR.mkdir(exist_ok=True)

# Supported image formats
SUPPORTED_FORMATS = {"jpg", "jpeg", "png", "webp", "bmp"}

# === CELL 4: Model Manager ===
class ModelManager:
    """Singleton class to manage the 3D model pipeline"""
    _instance = None
    _pipeline = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    async def get_pipeline(self):
        if self._pipeline is None:
            print("Loading Hunyuan3D pipeline...")
            try:
                # Import here to avoid issues if package isn't installed
                from hy3dgen.shapegen import Hunyuan3DDiTFlowMatchingPipeline
                self._pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(
                    'tencent/Hunyuan3D-2'
                )
                print("Pipeline loaded successfully")
            except Exception as e:
                logger.error(f"Failed to load pipeline: {e}")
                raise HTTPException(status_code=500, detail=f"Failed to load model: {e}")
        return self._pipeline

model_manager = ModelManager()

# === CELL 5: Utility Functions ===
def cleanup_file(file_path: str):
    """Background task to clean up temporary files"""
    try:
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Cleaned up: {file_path}")
    except Exception as e:
        logger.error(f"Error cleaning up {file_path}: {e}")

async def validate_image(file: UploadFile) -> None:
    """Validate uploaded image file"""
    if not file.filename:
        raise HTTPException(status_code=400, detail="No filename provided")

    file_ext = file.filename.split('.')[-1].lower()
    if file_ext not in SUPPORTED_FORMATS:
        raise HTTPException(
            status_code=400,
            detail=f"Unsupported format. Supported: {', '.join(SUPPORTED_FORMATS)}"
        )

    # Check file size (max 10MB)
    content = await file.read()
    if len(content) > 10 * 1024 * 1024:
        raise HTTPException(status_code=400, detail="File too large (max 10MB)")

    # Reset file position
    await file.seek(0)

async def save_upload_file(upload_file: UploadFile, destination: Path) -> None:
    """Save uploaded file"""
    content = await upload_file.read()
    async with aiofiles.open(destination, 'wb') as f:
        await f.write(content)

# === CELL 6: API Endpoints ===
@app.get("/")
async def root():
    """Health check endpoint"""
    return {
        "message": "3D Model Generator API (Colab Version)",
        "status": "healthy",
        "supported_formats": list(SUPPORTED_FORMATS),
        "temp_dir": str(TEMP_DIR)
    }

@app.get("/health")
async def health_check():
    """Detailed health check"""
    return {
        "status": "healthy",
        "pipeline_loaded": model_manager._pipeline is not None,
        "cuda_available": torch.cuda.is_available(),
        "device_count": torch.cuda.device_count() if torch.cuda.is_available() else 0,
        "temp_dir": str(TEMP_DIR)
    }

@app.post("/generate-3d")
async def generate_3d_model(
    background_tasks: BackgroundTasks,
    image: UploadFile = File(..., description="Image file to convert to 3D model")
):
    """Generate a 3D model from an uploaded image"""
    print("getted request for generat")
    await validate_image(image)

    request_id = str(uuid.uuid4())
    input_path = TEMP_DIR / f"input_{request_id}.jpg"
    output_path = TEMP_DIR / f"output_{request_id}.glb"

    try:
        print(f"Processing request {request_id}")

        # Save uploaded file
        await save_upload_file(image, input_path)
        print(f"Saved input: {input_path}")

        # Get pipeline
        pipeline = await model_manager.get_pipeline()

        # Generate 3D model
        print("Generating 3D model...")
        mesh_result = pipeline(image=str(input_path))

        if not mesh_result or len(mesh_result) == 0:
            raise HTTPException(status_code=500, detail="Failed to generate 3D model")

        mesh = mesh_result[0]

        # Convert to trimesh and export
        print("Converting to GLB...")
        trimesh_obj = trimesh.Trimesh(
            vertices=mesh.vertices,
            faces=mesh.faces,
            process=False
        )

        trimesh_obj.export(str(output_path))
        print(f"Model saved: {output_path}")

        # Schedule cleanup
        background_tasks.add_task(cleanup_file, str(input_path))
        background_tasks.add_task(cleanup_file, str(output_path))

        return FileResponse(
            path=str(output_path),
            media_type="model/gltf-binary",
            filename=f"model_{request_id}.glb"
        )

    except Exception as e:
        background_tasks.add_task(cleanup_file, str(input_path))
        background_tasks.add_task(cleanup_file, str(output_path))
        logger.error(f"Error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

# === CELL 7: Ngrok Setup ===
def setup_ngrok():
    """Setup ngrok tunnel"""
    try:
        # Set your ngrok auth token here
        ngrok.set_auth_token("your_ngrok_authtoken")

        # Kill any existing tunnels
        ngrok.kill()

        # Create new tunnel
        public_url = ngrok.connect(8000)
        print(f"🌐 Public URL: {public_url}")
        print(f"📋 API Docs: {public_url}/docs")
        print(f"🔍 Try it: {public_url}/")

        return public_url
    except Exception as e:
        print(f"Ngrok setup failed: {e}")
        return None

# === CELL 8: Run Server ===
server_running = False

def run_server():
    global server_running
    if server_running:
        print("⚠️ Server already running.")
        return None, None

    server_running = True

    import socket
    sock = socket.socket()
    sock.bind(('', 0))
    port = sock.getsockname()[1]
    sock.close()

    public_url = ngrok.connect(port)
    print(f"🌐 Public URL: {public_url}")
    print(f"📋 API Docs: {public_url}/docs")
    print(f"🔍 Try it: {public_url}/")

    config = uvicorn.Config(app=app, host="0.0.0.0", port=port, log_level="info")
    server = uvicorn.Server(config)

    def start_server():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(server.serve())

    thread = threading.Thread(target=start_server, daemon=True)
    thread.start()

    print("✅ Server started successfully!")
    return thread, public_url

# === CELL 9: Start the Server ===
# Run this cell to start the server
server_thread, public_url = run_server()

# === CELL 10: Test the API (Optional) ===
# You can use this cell to test your API
import requests
import json

def test_api(public_url):
    """Test the API endpoints"""
    try:
        # Test health check
        response = requests.get(f"{public_url}/health")
        print("Health Check:", response.json())

        # Test root endpoint
        response = requests.get(f"{public_url}/")
        print("Root Endpoint:", response.json())

        print("✅ API is working!")

    except Exception as e:
        print(f"❌ API test failed: {e}")

# Uncomment to test
# test_api(public_url)

# === CELL 11: Usage Instructions ===
print(f"""
🚀 Your 3D Model Generator API is now running!

📋 Usage Instructions:
1. Visit the public URL to see the API status
2. Go to /docs for interactive API documentation
3. Use /generate-3d endpoint to upload images and get 3D models

🔧 Endpoints:
- GET /         - API status
- GET /health   - Detailed health check
- POST /generate-3d - Generate 3D model from image

💡 Tips for Colab:
- Keep this notebook running to maintain the server
- The server runs in a background thread
- Files are automatically cleaned up after processing
- Use the public ngrok URL to access from anywhere

🐛 Troubleshooting:
- If ngrok fails, check your auth token
- If model loading fails, ensure all dependencies are installed
- Check GPU availability with /health endpoint
-public_url :{public_url}
""")

# === CELL 12: Stop Server (Optional) ===
# Run this cell to stop the server when done
def stop_server():
    """Stop the server and ngrok"""
    try:
        ngrok.kill()
        print("✅ Server stopped successfully!")
    except Exception as e:
        print(f"Error stopping server: {e}")

# Uncomment to stop
# stop_server()

INFO:     Started server process [5228]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:39373 (Press CTRL+C to quit)


🌐 Public URL: NgrokTunnel: "https://7c91906cd561.ngrok-free.app" -> "http://localhost:39373"
📋 API Docs: NgrokTunnel: "https://7c91906cd561.ngrok-free.app" -> "http://localhost:39373"/docs
🔍 Try it: NgrokTunnel: "https://7c91906cd561.ngrok-free.app" -> "http://localhost:39373"/
✅ Server started successfully!

🚀 Your 3D Model Generator API is now running!

📋 Usage Instructions:
1. Visit the public URL to see the API status
2. Go to /docs for interactive API documentation
3. Use /generate-3d endpoint to upload images and get 3D models

🔧 Endpoints:
- GET /         - API status
- GET /health   - Detailed health check
- POST /generate-3d - Generate 3D model from image

💡 Tips for Colab:
- Keep this notebook running to maintain the server
- The server runs in a background thread
- Files are automatically cleaned up after processing
- Use the public ngrok URL to access from anywhere

🐛 Troubleshooting:
- If ngrok fails, check your auth token
- If model loading fails, ensure all dependencies 

In [2]:
import requests

url = "https://7c91906cd561.ngrok-free.app/generate-3d"
image_path = "/content/Hunyuan3D-2/assets/empty-room-carpet-new-house-tx-usa-96362830.webp"

with open(image_path, "rb") as img_file:
    files = {"image": img_file}
    response = requests.post(url, files=files)

# Save the GLB model
with open("generated_model.glb", "wb") as out_file:
    out_file.write(response.content)

print("3D model saved as generated_model.glb")

getted request for generat
Processing request 855191a0-bc62-476d-9983-9a732f2599d9
Saved input: /content/temp_3d/input_855191a0-bc62-476d-9983-9a732f2599d9.jpg
Loading Hunyuan3D pipeline...


2025-07-09 11:54:15,797 - hy3dgen.shapgen - INFO - Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2/hunyuan3d-dit-v2-0
INFO:hy3dgen.shapgen:Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2/hunyuan3d-dit-v2-0
2025-07-09 11:54:15,802 - hy3dgen.shapgen - INFO - Model path not exists, try to download from huggingface
INFO:hy3dgen.shapgen:Model path not exists, try to download from huggingface
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

2025-07-09 11:54:17,791 - hy3dgen.shapgen - INFO - Loading model from /root/.cache/huggingface/hub/models--tencent--Hunyuan3D-2/snapshots/34e28261f71c32975727be8db0eace439a280f82/hunyuan3d-dit-v2-0/model.fp16.safetensors
INFO:hy3dgen.shapgen:Loading model from /root/.cache/huggingface/hub/models--tencent--Hunyuan3D-2/snapshots/34e28261f71c32975727be8db0eace439a280f82/hunyuan3d-dit-v2-0/model.fp16.safetensors


Pipeline loaded successfully
Generating 3D model...


Volume Decoding: 100%|██████████| 7134/7134 [02:22<00:00, 49.97it/s]


Converting to GLB...
Model saved: /content/temp_3d/output_855191a0-bc62-476d-9983-9a732f2599d9.glb
INFO:     34.125.215.160:0 - "POST /generate-3d HTTP/1.1" 200 OK
Cleaned up: /content/temp_3d/input_855191a0-bc62-476d-9983-9a732f2599d9.jpg
Cleaned up: /content/temp_3d/output_855191a0-bc62-476d-9983-9a732f2599d9.glb
3D model saved as generated_model.glb


# Make The mesh

In [ ]:
import torch
from hy3dgen.texgen import Hunyuan3DPaintPipeline
from hy3dgen.shapegen import Hunyuan3DDiTFlowMatchingPipeline
pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained('tencent/Hunyuan3D-2')
img_path="/content/room.jpeg"
mesh = pipeline(image=img_path)[0]
import trimesh
mesh = trimesh.Trimesh(
    vertices=mesh.vertices,
    faces=mesh.faces,
    colors=mesh.colors,
    process=False
)
# 3b. Save to disk (OBJ, PLY, GLTF, etc.)
mesh.export('output_mesh_glb.glb')   # or 'output_mesh.ply', 'output_mesh.glb', ...

# Clear The memory to save

In [ ]:
#!pip install numba
#from numba import cuda
#device = cuda.get_current_device()
#device.reset()

# Coloring the Mesh Don't run only if you has large gpu

In [ ]:
pip install bitsandbytes accelerate

In [ ]:
import torch
from hy3dgen.texgen import Hunyuan3DPaintPipeline
from hy3dgen.shapegen import Hunyuan3DDiTFlowMatchingPipeline
pipeline = Hunyuan3DPaintPipeline.from_pretrained('tencent/Hunyuan3D-2')
mesh = pipeline(mesh, image=img_path)


# Save The Mesh

In [ ]:
import trimesh
mesh = trimesh.Trimesh(
    vertices=mesh.vertices,
    faces=mesh.faces,
    #colors=mesh.colors,
    process=False
)
# 3b. Save to disk (OBJ, PLY, GLTF, etc.)
mesh.export('output_mesh_glb.glb')   # or 'output_mesh.ply', 'output_mesh.glb', ...

In [ ]:
# Install pymongo if you haven't already
!pip install pymongo

from pymongo import MongoClient

# Replace 'YOUR_MONGO_URI' with your actual MongoDB connection string
mongo_uri = 'YOUR_MONGO_URI'
client = MongoClient(mongo_uri)

# Replace 'your_database_name' and 'your_collection_name' with your actual database and collection names
db = client['your_database_name']
ai_servers_collection = db['your_collection_name']

def update_url(url):
  """Inserts a new URL into the MongoDB collection."""
  try:
    result = ai_servers_collection.insert_one({'url': url})
    print(f"New link added to DB with ID: {result.inserted_id}")
  except Exception as e:
    print(f"Error adding link to DB: {e}")

# Replace 'https://5e9a-35-240-130-158.ngrok-free.app' with the actual ngrok URL you want to save
ngrok_url = 'https://5e9a-35-240-130-158.ngrok-free.app'
update_url(ngrok_url)

# Close the connection (optional, good practice in scripts)
# client.close()